In [2]:
!pip install elasticsearch6 omegaconf


  Using cached elasticsearch6-6.8.2-py2.py3-none-any.whl.metadata (7.0 kB)
  Using cached omegaconf-2.3.0-py3-none-any.whl.metadata (3.9 kB)
  Using cached antlr4_python3_runtime-4.9.3-py3-none-any.whl
Using cached elasticsearch6-6.8.2-py2.py3-none-any.whl (90 kB)
Using cached omegaconf-2.3.0-py3-none-any.whl (79 kB)


In [3]:
import pycurl
import requests
import json
import pandas as pd
from tqdm import tqdm
from elasticsearch6 import Elasticsearch
import os
import re
import itertools
import numpy as np
from omegaconf import OmegaConf

### Функции

In [22]:
def get_ids_from_elastic_by_query(
    elastic_hosts: str, 
    elastic_index: str, 
    data_query: dict,
    size_for_save: int,
    path_for_save: str,
    request_size: int = 10_000
) -> list:
    '''
    Get data from elastic by user's query
    '''
    
    es = Elasticsearch(hosts=elastic_hosts)
    initial_request = es.search(index=elastic_index, scroll='10m', timeout='3s', size=request_size, body=data_query)
    my_scroll_id = initial_request['_scroll_id']
    num_of_entries = initial_request["hits"]["total"]
    all_data = []
    step_data = []
    for id_step in range(len(initial_request['hits']['hits'])):
        step_data.append(initial_request['hits']['hits'][id_step]['_id'])
    all_data.extend(step_data)
    file_number = 1
    for i in tqdm(range(int(num_of_entries/request_size))):
        request = es.scroll(scroll_id=my_scroll_id, scroll='10m')
        my_scroll_id = request['_scroll_id'] 
        step_data = []
        for id_step in range(len(request['hits']['hits'])):
            step_data.append(request['hits']['hits'][id_step]['_id'])
        all_data.extend(step_data)
        if len(all_data) >= size_for_save:
            file_name = f'{path_for_save}/data_from_elastic_{file_number}.parquet'
            print('Lets save some data, iter N ', file_number)
            result_df = pd.DataFrame(all_data, columns=['ids'])
            result_df.to_parquet(file_name)
            all_data = []
            file_number += 1
    file_name = f'{path_for_save}/data_from_elastic_{file_number}.parquet'
    print('Lets save some data, iter N ', file_number)
    result_df = pd.DataFrame(all_data, columns=['ids'])
    result_df.to_parquet(file_name)    

In [23]:
def glue_all_parquets_in_folder(path_to_folder, filename_for_save, delete_files=False):
    if delete_files == True:
        del_list = []
    files_list = []
    files_list += [
        each_file for each_file in os.listdir(path_to_folder) if each_file.endswith('.parquet')
    ]
    df = pd.DataFrame()
    for next_file in files_list:
        if next_file != filename_for_save:
            print(next_file)
            filename = f'{path_to_folder}/{next_file}'
            next_open = pd.read_parquet(filename)
            df = pd.concat((df, next_open), axis=0)
    df.to_parquet(f'{path_to_folder}/{filename_for_save}')
    if delete_files == True:
        for one_file in del_list:
            #print(one_file)
            os.remove(one_file)
    return df

# Получение данных товара по ID

In [4]:
def get_item_by_id(elastic_hosts: str, elastic_index: str, item_id: str) -> dict:
    es = Elasticsearch(hosts=elastic_hosts)
    data_query = {
        "query": {
            "ids": {
                "values": [item_id]
            }
        }
    }
    response = es.search(index=elastic_index, body=data_query)
    if response['hits']['total'] > 0:
        return response['hits']['hits'][0]
    else:
        return {"error": "Item not found"}


In [5]:
elastic_hosts = 'http://elastic-lb-01.prod.lan:19200/'
elastic_index = 'item_index_v7'
item_id = '100070534947' 

item_data = get_item_by_id(elastic_hosts, elastic_index, item_id)

if "error" not in item_data:
    print("Данные о товаре:")
    print(item_data['_source'])
else:
    print(item_data['error'])


Данные о товаре:
{'purpose': ['pdp'], 'long_web_name': 'Экошуба женский Batik 810-24о-3 серый 46-48/164', 'badges': [], 'logistic_class': '', 'logistic_class_code': 0, 'manufacturer_part_no': '810-24о-3', 'model': '810-24о-3', 'multiboxes_possible': '0', 'original_barcode': '', 'popularity': 1725353506660, 'registry_name': 'Batik Шуба укороченная с воротником (810-24о), размер 164/46-48, цвет Серый, бренд Batik', 'series': '', 'short_description': '', 'sub_brand': '', 'warranty_type': '', 'update_at': '2024-12-13 12:38:56', 'reviews_show': 1, 'successor': '', 'quality': 1.02, 'item_id': '100070534947', 'display_on_web': 1, 'single_offer_card': False, 'short_web_name': 'Экошуба женский Batik 810-24о-3 серый 46-48/164', 'prices': {'price_date': '2024-09-04 11:29:22', 'offers': [{'price': 13200, 'merchant_name': 'Batik (ДСМ)', 'merchant_id': 19010, 'type': 'Delivery', 'regions_2': 14797227486605312, 'regions_1': 4607182418800013312}, {'price': 13200, 'merchant_name': 'Batik', 'merchant_id

In [6]:
item_data['_source']

{'purpose': ['pdp'],
 'long_web_name': 'Экошуба женский Batik 810-24о-3 серый 46-48/164',
 'badges': [],
 'logistic_class': '',
 'logistic_class_code': 0,
 'manufacturer_part_no': '810-24о-3',
 'model': '810-24о-3',
 'multiboxes_possible': '0',
 'original_barcode': '',
 'popularity': 1725353506660,
 'registry_name': 'Batik Шуба укороченная с воротником (810-24о), размер 164/46-48, цвет Серый, бренд Batik',
 'series': '',
 'short_description': '',
 'sub_brand': '',
 'warranty_type': '',
 'update_at': '2024-12-13 12:38:56',
 'reviews_show': 1,
 'successor': '',
 'quality': 1.02,
 'item_id': '100070534947',
 'display_on_web': 1,
 'single_offer_card': False,
 'short_web_name': 'Экошуба женский Batik 810-24о-3 серый 46-48/164',
 'prices': {'price_date': '2024-09-04 11:29:22',
  'offers': [{'price': 13200,
    'merchant_name': 'Batik (ДСМ)',
    'merchant_id': 19010,
    'type': 'Delivery',
    'regions_2': 14797227486605312,
    'regions_1': 4607182418800013312},
   {'price': 13200,
    'me

In [ ]:
# price
item_data['_source']['prices']['final_price']

# название товара
item_data['_source']['long_web_name']

# бренд
item_data['_source']['brand']

# фото (несколько)
item_data['_source']['images']

# фото (несколько)
item_data['_source']['images'][0]['image_link']

In [13]:
product_attributes = [{item["name"]: item["value"]} for item in item_data['_source']['attributes']]
product_attributes

[{'Артикул производителя': ['810-24о-3']},
 {'Бренд': ['Batik']},
 {'Вид воротника': ['стойка']},
 {'Вид застежки': ['молния']},
 {'Вид меха': ['искусственный мех']},
 {'Вид принта': ['без принта']},
 {'Длина изделия': ['короткая']},
 {'Капюшон': ['без капюшона']},
 {'Коллекция': ['базовая коллекция 24']},
 {'Комплектность': ['шуба 1 шт']},
 {'Материал': ['искусственный мех']},
 {'Материал подкладки': ['pongee']},
 {'Модель': ['810-24о-3']},
 {'Особенности': ['пушистые']},
 {'Пол': ['женский']},
 {'Размер RU': ['46; 48']},
 {'Размер производителя': ['46-48/164']},
 {'Сезон': ['демисезон']},
 {'Силуэт': ['свободный']},
 {'Состав ткани': ['100% экомех']},
 {'Стиль': ['повседневный']},
 {'Температурный режим': ['от +5 С до -20 С']},
 {'Тип': ['экошуба']},
 {'Уход': ['Следуйте рекомендациям по уходу, указанным на этикетке изделия']},
 {'Цвет': ['серый']},
 {'Цвет производителя': ['серый']}]

### Текст товара Elastic

In [7]:
{'purpose': ['pdp'], 
 'long_web_name': 'Шуба женская Batik 810-24о-3 серая 46-48/164', 
 'badges': [], 
 'logistic_class': '', 
 'logistic_class_code': 0, 
 'manufacturer_part_no': '810-24о-3', 
 'model': '810-24о-3', 
 'multiboxes_possible': '0', 
 'original_barcode': '', 
 'popularity': 1725353506660, 
 'registry_name': 'Batik Шуба укороченная с воротником (810-24о), размер 164/46-48, цвет Серый, бренд Batik', 
 'series': '', 
 'short_description': '', 
 'sub_brand': '', 
 'warranty_type': '', 
 'update_at': '2024-10-04 13:07:42', 
 'reviews_show': 1, 
 'successor': '', 
 'quality': 1.02, 
 'item_id': '100070534947', 
 'display_on_web': 1, 
 'single_offer_card': False, 
 'short_web_name': 'Шуба женская Batik 810-24о-3 серая 46-48/164', 
 'prices': {'price_date': '2024-09-04 11:29:22', 
            'offers': [{'merchant_id': 19010, 
                         'price': 13200, 
                        'type': 'Delivery', 
                        'merchant_name': 'Batik (ДСМ)', 
                        'regions_1': 4607182418800013312, 
                        'regions_2': 14797227486605312}, 
                       
                       {'merchant_id': 59595, 
                        'price': 13200, 
                        'type': 'Delivery', 
                        'merchant_name': 'Batik', 
                        'regions_1': 3454260912012128256, 
                        'regions_2': 1268836351340544}, 
                       
                       {'merchant_id': 131915, 
                        'price': 13200, 
                        'type': 'Delivery', 
                        'merchant_name': 'Batik Novosibirsk', 
                        'regions_1': 3454260912012128256, 
                        'regions_2': 1268836351340544}, 
                       
                       {'merchant_id': 131916, 
                        'price': 13200, 
                        'type': 'Delivery', 
                        'merchant_name': 'Batik msk', 
                        'regions_1': 3454260914159611904, 
                        'regions_2': 1268836351340544}, 
                       
                       {'merchant_id': 220078, 
                        'price': 24890, 
                        'type': 'Delivery', 
                        'merchant_name': 'Дом сумок FBS', 
                        'regions_1': 3454260914159611904, 
                        'regions_2': 1268836351340544}], 
            'bonus': {'k': 0, 'bonus_date': '2024-11-26 02:07:23', 'merchant_bonus': {'59595': {'k': 46, 'merchant_id': '59595', 'merchant_name': 'Batik', 'bonus_date': '2024-11-26 01:08:35'}, '131915': {'k': 25, 'merchant_id': '131915', 'merchant_name': 'Batik Novosibirsk', 'bonus_date': '2024-11-26 02:07:22'}, '131916': {'k': 25, 'merchant_id': '131916', 'merchant_name': 'Batik msk', 'bonus_date': '2024-11-26 02:07:23'}}}, 
            'extra': {'bpg2_competitors': 0}, 
            'is_available': 1, 
            'last_price': 13200, 
            'final_price': 13200, 
            'crossed_price': None, 
            'crossed_price_term': '', 
            'min_price': 13200, 
            'max_price': 13200, 
            'total_offers': 2, 
            'discounts': [], 
            'favorite_offer': {'offer_id': '18335138507', 'price': 13200, 'partner_id': '000016365', 'contract_id': '000000000072767', 'merchant_id': '59595', 'merchant_name': 'Batik', 'old_price': None}, 
            'price_history': [], 
            'delivery_date': '2024-09-06 00:00:00', 
            'price_change_percentage': None, 
            'price_adjustments': []}, 
 'offers_v2': [{'merchantId': 19010, 
                'region': [7, 5, 26, 6, 9, 20, 15, 13, 59, 16, 18, 64, 21, 56, 52, 63, 2, 12, 43, 73, 58, 66, 72, 86, 89, 45, 74, 70, 19, 55, 24, 22, 42, 75, 3, 38, 17, 54, 4, 79, 14, 27, 25, 49, 28, 65, 29, 35, 11, 83, 60, 39, 10, 78, 53, 51, 47, 34, 1, 23, 30, 61, 92, 8, 91, 48, 46, 31, 44, 57, 32, 68, 76, 50, 62, 71, 36, 33, 69, 37, 40, 77, 67], 
                'price': 13200, 
                'type': 'Delivery', 
                'service_scheme': 'DELIVERY_BY_MERCHANT', 
                'favorite_price_for_regions': [14, 49, 65, 83, 91, 92], 
                'favorite': False, 
                'group': 1, 
                'old_price': None}, 
               
               {'merchantId': 59595, 'region': [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 15, 16, 17, 18, 19, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 42, 43, 44, 45, 46, 47, 48, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 86, 89], 'price': 13200, 'type': 'Delivery', 'service_scheme': 'DELIVERY_BY_GOODS', 'favorite_price_for_regions': [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 15, 16, 17, 18, 19, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 42, 43, 44, 45, 46, 47, 48, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 86, 89], 'favorite': True, 'group': 1, 'old_price': None}, 
               
               {'merchantId': 131915, 'region': [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 15, 16, 17, 18, 19, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 42, 43, 44, 45, 46, 47, 48, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 86, 89], 'price': 13200, 'type': 'Delivery', 'service_scheme': 'DELIVERY_BY_GOODS', 'favorite_price_for_regions': None, 'favorite': False, 'group': 1, 'old_price': None}, 
               
               {'merchantId': 131916, 'region': [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 42, 43, 44, 45, 46, 47, 48, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 86, 89], 'price': 13200, 'type': 'Delivery', 'service_scheme': 'DELIVERY_BY_GOODS', 'favorite_price_for_regions': None, 'favorite': False, 'group': 1, 'old_price': None}, 
               
               {'merchantId': 220078, 'region': [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 42, 43, 44, 45, 46, 47, 48, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 86, 89], 'price': 24890, 'type': 'Delivery', 'service_scheme': 'DELIVERY_BY_GOODS', 'favorite_price_for_regions': [20], 'favorite': False, 'group': 1, 'old_price': None}], 
 'brand': 'Batik', 
 'brand_slug': 'batik', 
 'category': {'master': [['17', '1701', '170101', '17010103', '1701010301', '170101030101']], 
              'sandbox': [], 
              'web': [['2463684633831608', '2463684633831610', '2463684633831611', '2463684633831664']]}, 
 'country_of_origin': '', 
 'gtins': ['4640136175447'], 
 'logistic_class_type': 0, 
 'long_description': 'Шуба укороченная с воротником на молнии. Выполнена из экомеха с подкладом и утеплителем. Легкая и стильная шубка "под норку", которая станет отличным приобретением на осень и зиму для девушек-подростков и взрослых. Расцветка серая однотонная.  <br> <h4>Характеристики:</h4> <ul> <li>Ткань верха: Экомех </li> <li>Подклад: pongee</li> <li>Утеплитель: 60 г/м?</li> <li>Температурный режим от -5 С до +15 С с поддевой</li> <li>Производитель: batik</li> </ul>', 'nds': '20', 'operation': 'create', 'slug': 'shuba-zhenskaya-batik-810-24o-3-seraya-46-48-164-100070534947', 
 'toplist': 0, 
 'warranty': '', 
 'attributes': [{'units': '', 
                 'name': 'Артикул производителя', 
                 'group': 'Заводские данные о товаре', 
                 'sequence': 1, 
                 'purpose': ['filters', 'print', 'weblong'], 
                 'type': 'character string', 
                 'feature_description': '', 
                 'value': ['810-24о-3']}, 
                
                {'units': '', 'name': 'Бренд', 'group': 'Заводские данные о товаре', 'sequence': 1, 'purpose': ['filters', 'print', 'weblong'], 'type': 'character string', 'feature_description': '', 'value': ['Batik']}, 
                
                {'units': '', 'name': 'Вид застежки', 'group': 'Дополнительная информация', 'sequence': 2147483647, 'purpose': ['filters', 'filtersshort', 'print', 'weblong'], 'type': 'character string', 'feature_description': '', 'value': ['молния']}, 
                
                {'units': '', 'name': 'Вид меха', 'group': 'Дополнительная информация', 'sequence': 2147483647, 'purpose': ['filters', 'filtersshort', 'print', 'weblong'], 'type': 'character string', 'feature_description': '', 'value': ['искусственный мех']}, 
                
                {'units': '', 'name': 'Вид принта', 'group': 'Материалы и цвет', 'sequence': 2147483647, 'purpose': ['filters', 'filtersshort', 'print', 'weblong'], 'type': 'character string', 'feature_description': '', 'value': ['без принта']}, 
                
                {'units': '', 'name': 'Длина', 'group': 'Дополнительная информация', 'sequence': 2147483647, 'purpose': ['filters', 'filtersshort', 'print', 'weblong'], 'type': 'character string', 'feature_description': '', 'value': ['короткая']}, 
                
                {'units': '', 'name': 'Коллекция', 'group': 'Дополнительная информация', 'sequence': 2147483647, 'purpose': ['print'], 'type': 'character string', 'feature_description': '', 'value': ['базовая коллекция 24']}, 
                
                {'units': '', 'name': 'Комплектность', 'group': 'Дополнительная информация', 'sequence': 2147483647, 'purpose': ['print'], 'type': 'character string', 'feature_description': '', 'value': ['шуба 1 шт']}, 
                
                {'units': '', 'name': 'Материал', 'group': 'Материалы и цвет', 'sequence': 2147483647, 'purpose': ['filters', 'filtersshort', 'print', 'weblong', 'webshort'], 'type': 'character string', 'feature_description': '', 'value': ['искусственный мех']}, 
                
                {'units': '', 'name': 'Модель', 'group': 'Заводские данные о товаре', 'sequence': 1, 'purpose': ['filters', 'print', 'weblong'], 'type': 'character string', 'feature_description': '', 'value': ['810-24о-3']}, 
                
                {'units': '', 'name': 'Особенности', 'group': 'Дополнительная информация', 'sequence': 2147483647, 'purpose': ['filters', 'filtersshort', 'print', 'weblong'], 'type': 'character string', 'feature_description': '', 'value': ['экошуба']}, 
                
                {'units': '', 'name': 'Пол', 'group': 'Основные характеристики', 'sequence': 2147483647, 'purpose': ['filters', 'filtersshort', 'print', 'weblong'], 'type': 'character string', 'feature_description': '', 'value': ['женский']}, 
                {'units': '', 'name': 'Размер RU', 'group': 'Основные характеристики', 'sequence': 2147483647, 'purpose': ['filters', 'filtersshort', 'print', 'weblong'], 'type': 'character string', 'feature_description': '', 'value': ['46; 48']}, 
                {'units': '', 'name': 'Размер производителя', 'group': 'Основные характеристики', 'sequence': 2147483647, 'purpose': ['filters', 'filtersshort', 'print', 'weblong'], 'type': 'character string', 'feature_description': '', 'value': ['46-48/164']}, 
                {'units': '', 'name': 'Сезон', 'group': 'Основные характеристики', 'sequence': 2147483647, 'purpose': ['filters', 'filtersshort', 'print', 'weblong'], 'type': 'character string', 'feature_description': '', 'value': ['демисезон']}, 
                {'units': '', 'name': 'Силуэт', 'group': 'Дополнительная информация', 'sequence': 2147483647, 'purpose': ['filters', 'filtersshort', 'print', 'weblong'], 'type': 'character string', 'feature_description': '', 'value': ['прямой']}, 
                {'units': '', 'name': 'Состав ткани', 'group': 'Материалы и цвет', 'sequence': 2147483647, 'purpose': ['filters', 'filtersshort', 'print', 'weblisting', 'weblong'], 'type': 'character string', 'feature_description': '', 'value': ['100% экомех']}, 
                {'units': '', 'name': 'Стиль', 'group': 'Дополнительная информация', 'sequence': 2147483647, 'purpose': ['filters', 'filtersshort', 'print', 'weblong'], 'type': 'character string', 'feature_description': '', 'value': ['повседневный']}, 
                {'units': '', 'name': 'Тип', 'group': 'Основные характеристики', 'sequence': 2147483647, 'purpose': ['filters', 'filtersshort', 'print', 'weblong'], 'type': 'character string', 'feature_description': '', 'value': ['шуба']}, 
                {'units': '', 'name': 'Уход', 'group': 'Материалы и цвет', 'sequence': 2147483647, 'purpose': ['filters', 'filtersshort', 'print', 'weblong'], 'type': 'character string', 'feature_description': '', 'value': ['Следуйте рекомендациям по уходу, указанным на этикетке изделия']}, 
                {'units': '', 'name': 'Цвет', 'group': 'Материалы и цвет', 'sequence': 2147483647, 'purpose': ['filters', 'filtersshort', 'print', 'weblong'], 'type': 'character string', 'feature_description': '', 'value': ['серый']}, 
                {'units': '', 'name': 'Цвет производителя', 'group': 'Материалы и цвет', 'sequence': 2147483647, 'purpose': ['filters', 'filtersshort', 'print', 'weblong'], 'type': 'character string', 'feature_description': '', 'value': ['серый']}], 
 'video': '', 
 'images': [{'image_link': '/hlr-system/453/975/693/011/58/100070534947b0.jpg', 'image_order': '1'}, 
            {'image_link': '/hlr-system/453/975/793/011/58/100070534947b1.jpg', 'image_order': '2'}, 
            {'image_link': '/hlr-system/453/975/893/011/58/100070534947b2.jpg', 'image_order': '3'}, 
            {'image_link': '/hlr-system/453/975/993/011/58/100070534947b3.jpg', 'image_order': '4'}, 
            {'image_link': '/hlr-system/453/976/093/011/58/100070534947b4.jpg', 'image_order': '5'}, 
            {'image_link': '/hlr-system/453/976/193/011/58/100070534947b5.jpg', 'image_order': '6'}, 
            {'image_link': '/hlr-system/453/976/293/011/58/100070534947b6.jpg', 'image_order': '7'}, 
            {'image_link': '/hlr-system/453/976/393/011/58/100070534947b7.jpg', 'image_order': '8'}, 
            {'image_link': '/hlr-system/453/976/493/011/58/100070534947b8.jpg', 'image_order': '9'}, 
            {'image_link': '/hlr-system/453/976/593/011/58/100070534947b9.jpg', 'image_order': '10'}, 
            {'image_link': '/hlr-system/140/732/515/930/115/8/100070534947b10.jpg', 'image_order': '11'}, 
            {'image_link': '/hlr-system/140/732/516/930/115/8/100070534947b11.jpg', 'image_order': '12'}, 
            {'image_link': '/hlr-system/140/732/517/930/115/8/100070534947b12.jpg', 'image_order': '13'}], 
 'thumbnail': '', 
 'documents': [], 
 'boxes': [{'box': '1', 'packaging_unit': 'см', 'width': 15, 'height': 53, 'length': 33, 'weight_unit': 'кг', 'weight': 2}], 
 'flags': ['DeliveryAvailable', 'IsBPG1Available'], 
 'collections': [12782, 692769, 692774, 692791, 692844, 692845, 692846, 728227, 728229, 728230, 728884, 728890, 879467, 1036786, 1036810, 1119159, 3981113, 3981119, 4025384, 4034310, 4328612, 6248362, 6683436, 7416831, 7424608, 8799151, 8906384, 9228205, 9289256, 9309548, 9331631, 9345085, 9345086, 9345385, 9345387, 9345395, 9345401, 9353576, 9353740, 9359973, 9359974, 9374254, 9382508, 9434684, 9434685, 9434687, 9434688, 9460896, 9460926, 9464232], 
 'main_web_collection_id': 4034310, 
 'url': '/catalog/details/shuba-zhenskaya-batik-810-24o-3-seraya-46-48-164-100070534947/', 
 'reviews_count': 2, 
 'reviews_rating': 5, 
 'extra': {'attributes_on_page': 20, 'is_attributes_over_page': 1}, 
 'deputy': {'rule_id': '373', 'fraction_id': '11577DD29C182A33C48C105FABAC90B1', 'deputy_id': '44E45F73193E67EF05A59055214AE771', 'selector_plp': {'name': 'Размер RU', 'control_type': 'VALUE', 'value': '46; 48'}, 'selector_pdp': [{'name': 'Размер RU', 'control_type': 'VALUE', 'value': '46; 48 @46-48/164'}, {'name': 'Цвет', 'control_type': 'IMAGES', 'value': 'серый'}]}, 
 'logistic_tags': ['ANYPVZ', 'ANYPVZ20', 'BORSCH15', 'DSM', 'DSMVW', 'FF', 'MGT', 'MGT20', 'REGFULL30', 'SAMOKAT', 'SBLM135', 'SBLM150', 'SBLM30', 'SBLMMGT_13X', 'SBLMMGT_AVIA', 'SHIP25_20'], 
 'content_flags': ['fitting', 'honest_sign', 'fashion'], 
 'publication_date': '2024-09-03 11:51:45', 
 'partition_id': 199, 
 'deputy_id': '44E45F73193E67EF05A59055214AE771', 
 'nested_rating': [{'collection_id': '12782', 'value': 580697, 'value_bi': None}, {'collection_id': '4025384', 'value': 129271, 'value_bi': None}, {'collection_id': '4034310', 'value': 100571, 'value_bi': None}, {'collection_id': '4328612', 'value': 107855, 'value_bi': None}, {'collection_id': '6248362', 'value': 107102, 'value_bi': None}, {'collection_id': '6683436', 'value': 1237608, 'value_bi': None}, {'collection_id': '728227', 'value': 579281, 'value_bi': None}, {'collection_id': '728229', 'value': 238811, 'value_bi': None}, {'collection_id': '728230', 'value': 114745, 'value_bi': None}, {'collection_id': '728884', 'value': 105099, 'value_bi': None}, {'collection_id': '728890', 'value': 100787, 'value_bi': None}, {'collection_id': '8799151', 'value': 949064, 'value_bi': None}, {'collection_id': '8906384', 'value': 1561416, 'value_bi': None}, {'collection_id': '9289256', 'value': 100092, 'value_bi': None}, {'collection_id': '9309548', 'value': 104775, 'value_bi': None}, {'collection_id': '9345085', 'value': 102498, 'value_bi': None}, {'collection_id': '9345086', 'value': 160871, 'value_bi': None}, {'collection_id': '9345385', 'value': 157846, 'value_bi': None}, {'collection_id': '9353576', 'value': 132779, 'value_bi': None}, {'collection_id': '9359973', 'value': 123650, 'value_bi': None}, {'collection_id': '9359974', 'value': 123650, 'value_bi': None}, {'collection_id': '9374254', 'value': 123650, 'value_bi': None}, {'collection_id': '9382508', 'value': 123892, 'value_bi': None}, {'collection_id': '9434684', 'value': 106709, 'value_bi': None}, {'collection_id': '9434685', 'value': 106709, 'value_bi': None}, {'collection_id': '9434687', 'value': 106709, 'value_bi': None}, {'collection_id': '9434688', 'value': 106709, 'value_bi': None}], 
 'nested_filters': [{'id': '0611F8B22C8DFC2F0D76ABADA8D18D7A', 'string_values': ['серый'], 'float_values': None}, {'id': '0F60EA740B308E674F81A1F6A605C3F9', 'string_values': ['шуба'], 'float_values': None}, {'id': '2AA9A3AE46E6485298BE5B23F6360C9B', 'string_values': ['да'], 'float_values': None}, {'id': '2B0B1FF4756D49CF84B094522D57ED3D', 'string_values': ['Batik'], 'float_values': None}, {'id': '36767696AC0103B8B968B351BADD306F', 'string_values': ['женский'], 'float_values': None}, {'id': '42A5DD0480593572482A3105FCCDB543', 'string_values': ['46', '48'], 'float_values': None}, {'id': '4CB2C27EAAFC4EB39378C4B7487E6C9E', 'string_values': ['да'], 'float_values': None}, {'id': '830DF06A4ECDDABB5AB8563097C4D894', 'string_values': ['46', '48'], 'float_values': None}, {'id': '88C83F68482F447C9F4E401955196697', 'string_values': None, 'float_values': [13200]}, {'id': 'A03364050801C25CD0A856C734F74FE9', 'string_values': ['да'], 'float_values': None}, {'id': 'A3B6D6988B9310C6B5F8A54E2C28CE15', 'string_values': ['демисезон'], 'float_values': None}, {'id': 'EED496DC7FDD56F0B450A4B64BD0699B', 'string_values': ['искусственный мех'], 'float_values': None}, {'id': 'F7DB604EDF6D4C42920DF290E6AF8716', 'string_values': ['нет'], 'float_values': None}, {'id': 'FC7D537CF7CD433DA82630BA69606666', 'string_values': ['нет'], 'float_values': None}, {'id': 'FE476AD4A46D4F8EA7B1AB7F30ED017S', 'string_values': ['Batik (ДСМ)', 'Batik', 'Batik Novosibirsk', 'Batik msk', 'Дом сумок FBS'], 'float_values': None}]}



{'purpose': ['pdp'],
 'long_web_name': 'Шуба женская Batik 810-24о-3 серая 46-48/164',
 'badges': [],
 'logistic_class': '',
 'logistic_class_code': 0,
 'manufacturer_part_no': '810-24о-3',
 'model': '810-24о-3',
 'multiboxes_possible': '0',
 'original_barcode': '',
 'popularity': 1725353506660,
 'registry_name': 'Batik Шуба укороченная с воротником (810-24о), размер 164/46-48, цвет Серый, бренд Batik',
 'series': '',
 'short_description': '',
 'sub_brand': '',
 'warranty_type': '',
 'update_at': '2024-10-04 13:07:42',
 'reviews_show': 1,
 'successor': '',
 'quality': 1.02,
 'item_id': '100070534947',
 'display_on_web': 1,
 'single_offer_card': False,
 'short_web_name': 'Шуба женская Batik 810-24о-3 серая 46-48/164',
 'prices': {'price_date': '2024-09-04 11:29:22',
  'offers': [{'merchant_id': 19010,
    'price': 13200,
    'type': 'Delivery',
    'merchant_name': 'Batik (ДСМ)',
    'regions_1': 4607182418800013312,
    'regions_2': 14797227486605312},
   {'merchant_id': 59595,
    'pr

# Получение данных НЕСКОЛЬКИХ товаров по ID

In [8]:
def get_items_by_ids(elastic_hosts: str, elastic_index: str, item_ids: list) -> list:
    from elasticsearch6 import Elasticsearch
    es = Elasticsearch(hosts=elastic_hosts)
    data_query = {
        "query": {
            "ids": {
                "values": item_ids
            }
        }
    }
    response = es.search(index=elastic_index, body=data_query)
    if response['hits']['total'] > 0:
        return [hit['_source'] for hit in response['hits']['hits']]
    else:
        return []


In [9]:
item_ids = ['100027371781', '600015749679', '100070722124'] 
items_data = get_items_by_ids(elastic_hosts, elastic_index, item_ids)

if items_data:
    print("Данные о товарах:")
    for item in items_data:
        print(item)
else:
    print("Товары не найдены")


Данные о товарах:
{'purpose': ['pdp', 'plp'], 'long_web_name': 'Стул для кухни Stool Group DSW Style V белый, 3 шт', 'badges': [], 'logistic_class': '', 'logistic_class_code': 0, 'manufacturer_part_no': 'Y801-V SEAT white BOX3', 'model': 'EAMES DSW Style V', 'multiboxes_possible': '0', 'original_barcode': '', 'popularity': 1704067200, 'registry_name': 'Стул для кухни обеденный DSW Style V белый, разборный фрейм (комплект 3 стула)', 'series': 'DSW Style V', 'short_description': '', 'sub_brand': '', 'warranty_type': '', 'update_at': '2024-11-22 17:09:12', 'reviews_show': 1, 'successor': '', 'quality': 1.56, 'item_id': '600015749679', 'display_on_web': 1, 'single_offer_card': False, 'short_web_name': 'Стул для кухни Stool Group DSW Style V белый, 3 шт', 'prices': {'price_date': '2024-09-04 03:45:17', 'offers': [{'merchant_id': 102123, 'price': 5490, 'type': 'Delivery', 'merchant_name': 'Stool Group (FBS)', 'regions_1': 3454260914159611904, 'regions_2': 1268836351340544}], 'bonus': {'k': 0

In [10]:
print(type(items_data))

<class 'list'>
